In [2]:
!pip install gradio tensorflow pillow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.0 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [3]:
import tensorflow_hub as hub
import tensorflow as tf
import numpy as np
from PIL import Image
import gradio as gr
import requests

# Descargar el mapa de clases de ImageNet
imagenet_classes_url = "https://storage.googleapis.com/download.tensorflow.org/data/imagenet_class_index.json"
response = requests.get(imagenet_classes_url)
imagenet_classes = response.json()

# Cargar el modelo MobileNetV2 desde TensorFlow Hub
model = hub.KerasLayer("https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/classification/5")

# Procesar la imagen
def process_image(image):
    img = image.resize((224, 224))  # Redimensionar al tamaño esperado
    img_array = np.array(img) / 255.0  # Normalizar los valores de píxeles
    return np.expand_dims(img_array, axis=0)  # Añadir una dimensión para batch

# Predicción
def predict_image(image):
    img = process_image(image)
    prediction = model(img)  # Realizar la predicción con TensorFlow Hub
    predicted_index = np.argmax(prediction[0].numpy())  # Índice de la clase con mayor probabilidad
    class_label = imagenet_classes[str(predicted_index)][1]  # Etiqueta de la clase
    confidence = prediction[0][predicted_index].numpy()  # Confianza de la predicción
    return f"Raza/Clase predicha: {class_label}", f"Confianza: {confidence * 100:.2f}%"

# Función para la interfaz
def welcome_and_predict(image):
    if image is None:
        return "¡Bienvenido! Por favor, sube una imagen para empezar.", ""
    return predict_image(image)

# Crear la interfaz con Gradio
with gr.Blocks() as demo:
    gr.Markdown("## ¡Bienvenido al Reconocedor de Razas de Perros 🐶!")
    gr.Markdown("Sube una foto de tu perro y obtendrás una predicción de su raza.")

    with gr.Row():
        img_input = gr.Image(type="pil", label="Carga tu foto aquí")
        output_label = gr.Textbox(label="Resultado")
        output_confidence = gr.Textbox(label="Confianza")

    submit_btn = gr.Button("¡Predecir!")
    clear_btn = gr.Button("Cancelar")

    # Conectar funciones
    submit_btn.click(welcome_and_predict, inputs=img_input, outputs=[output_label, output_confidence])
    clear_btn.click(lambda: (None, ""), inputs=[], outputs=[img_input, output_label, output_confidence])

# Lanzar la aplicación
demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://058f2df85be0388e09.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
